# Fast Tree of DADA2 ASVs
* Tree building from alignment using DADA2 sequences from three hyphosphere experiments.  
* Fasta has already been thresholded to reduce computational demand
* Set kernel to Python 2
* This version attempts to keep archaeal sequences

In [13]:
workDir = '/home/be68/Hyphosphere/data/3Exp/Fasttree/AllDomain/'
OTUFileDir = '/home/be68/Hyphosphere/data/3Exp/'
OTUFile = '/home/be68/Hyphosphere/data/3Exp/phyloseq/seqs_thresh.fasta'


nprocs = 20

In [14]:
import os
import numpy as np
#import entrez.direct
from cogent.app.fasttree import build_tree_from_alignment
from cogent import DNA, LoadSeqs
from Bio import Entrez, Phylo
Entrez.email = "be68@cornell.edu"
#from IPython.display import display, Image, SVG
#from cogent3 import LoadSeqs, FastTree


In [3]:
#%load_ext rpy2.ipython

In [15]:
if not os.path.isdir(workDir):
    os.mkdir(workDir)

In [19]:
!cd $workDir

In [17]:
!pwd

/data/home/be68/Hyphosphere/notebooks/MiSeqRuns


In [20]:
!cd $workDir; ln -f -s $OTUFile

In [21]:
!cd $workDir; head $OTUFile

>ASV1
TACGAAGGGGGCTAGCGTTGTTCGGATTTACTGGGCGTAAAGCGCACGTAGGCGGACTATTAAGTCAGGGGTGAAATCCCGGGGCTCAACCCCGGAACTGCCTTTGATACTGGTAGTCTTGAGTTCGAGAGAGGTGAGTGGAATTCCGAGTGTAGAGGTGAAATTCGTAGATATTCGGAGGAACACCAGTGGCGAAGGCGGCTCACTGGCTCGATACTGACGCTGAGGTGCGAAAGCGTGGGGAGCAAACAGG
>ASV2
GACATAGGTGGCAAACATTATCCGGAATTATTGGGCGTAAAGGGTGCGTAGGCGGCATGATAAGTTGCTGGTGGGAAATCAAGGCTCAACCTTGTGGAAGCTAGCAATACTGTCAAGCTAGAGGGCAGAAGAGGTTAACGGAACTCTATGTGGAGCGGTAAAATGTGTAGATATATAGAAGAACATCAATAAAGGCGAAGGCAGTTAACTAGTCTGTCCCTGACGTTGAGGCACGAAAGCGTGGGGAGCAAAACGG
>ASV3
TACGGAGGGAGCTAGCGTTGTTCGGAATTACTGGGCGTAAAGCGCACGTAGGCGGCTTTGTAAGTCAGAGGTGAAAGCCTGGAGCTCAACTCCAGAACTGCCTTTGAGACTGCATCGCTTGAATCCAGGAGAGGTGAGTGGAATTCCGAGTGTAGAGGTGAAATTCGTAGATATTCGGAAGAACACCAGTGGCGAAGGCGGCTCACTGGACTGGTATTGACGCTGAGGTGCGAAAGCGTGGGGAGCAAACAGG
>ASV4
GACATAGGTGGCGAACGTTATCCGGAATTATTGGGCGTAAAGGATGCGTAGATGGCAGAGTAAGTTACTGGTTGATGTCAAACTCAATTTGACGGAAGCTGGTAATACTGTTTTGCTAGAGGACAGGAGAGGTTGATGGAATTCTGTGTGGAGCGGTGAAATGCGTTGATCTACAGAGGAACACCAAAAAAGGCGAAGGCAGTCAACTATC

In [22]:
!printf "Number of OTUs in fasta: "
!cd $workDir; grep -c ">" $OTUFile

Number of OTUs in fasta: 15600


## Using SSU-Align to align seqs and masking based on alignment posterior probabilities.

In [30]:
!cd $workDir; ssu-prep -n bacteria -f -x -b 50 --rfonly --dna $OTUFile ssu_aln 20

# _ssu-prep :: prepare SSU rRNA sequences for parallel ssu-align jobs
# SSU-ALIGN 0.1.1 (Feb 2016)
# Copyright (C) 2016 Howard Hughes Medical Institute
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# command: _ssu-prep -x -f -b 50 -n bacteria --dna --rfonly /home/be68/Hyphosphere/data/3Exp/phyloseq/seqs_thresh.fasta ssu_aln 20
# date:    Tue Sep 24 08:45:17 2019
#
# Validating input sequence file ... done.
#
# Preparing 20 ssu-align jobs ...
# Partitioning seqs with goal of equalizing total number of nucleotides per job ...
#
# output file name      description                                           
# --------------------  ------------------------------------------------------
  ssu_aln/seqs_thresh.fasta.1  partition  1 FASTA sequence file (781 seqs; 197638 nt)
  ssu_aln/seqs_thresh.fasta.2  partition  2 FASTA sequence file (781 seqs; 197659 nt)
  ssu_aln/seqs_thresh.fasta.3  partition  3 FASTA sequ

In [31]:
!cd $workDir; ./ssu_aln.ssu-align.sh

# Executing: ssu-align -b 50 -n bacteria --dna --rfonly ssu_aln/seqs_thresh.fasta.1 ssu_aln/ssu_aln.1 > /dev/null &
# Executing: ssu-align -b 50 -n bacteria --dna --rfonly ssu_aln/seqs_thresh.fasta.2 ssu_aln/ssu_aln.2 > /dev/null &
# Executing: ssu-align -b 50 -n bacteria --dna --rfonly ssu_aln/seqs_thresh.fasta.3 ssu_aln/ssu_aln.3 > /dev/null &
# Executing: ssu-align -b 50 -n bacteria --dna --rfonly ssu_aln/seqs_thresh.fasta.4 ssu_aln/ssu_aln.4 > /dev/null &
# Executing: ssu-align -b 50 -n bacteria --dna --rfonly ssu_aln/seqs_thresh.fasta.5 ssu_aln/ssu_aln.5 > /dev/null &
# Executing: ssu-align -b 50 -n bacteria --dna --rfonly ssu_aln/seqs_thresh.fasta.6 ssu_aln/ssu_aln.6 > /dev/null &
# Executing: ssu-align -b 50 -n bacteria --dna --rfonly ssu_aln/seqs_thresh.fasta.7 ssu_aln/ssu_aln.7 > /dev/null &
# Executing: ssu-align -b 50 -n bacteria --dna --rfonly ssu_aln/seqs_thresh.fasta.8 ssu_aln/ssu_aln.8 > /dev/null &
# Executing: ssu-align -b 50 -n bacteria --dna --rfonly ssu_aln/seqs_thr

# Cleaning up by removing original files that were just merged ... done.
#
# List of executed commands saved in:     ssu_aln.ssu-merge.log.
# Output printed to the screen saved in:  ssu_aln.ssu-merge.sum.
#
# All output files created in directory ./ssu_aln/
#
# CPU time:  00:00:00.60  (hh:mm:ss)
# 


In [32]:
!cd $workDir; ssu-mask --pf 0.95 --pt 0.95 --dna --afa  ssu_aln

# _ssu-mask :: mask SSU rRNA alignments
# SSU-ALIGN 0.1.1 (Feb 2016)
# Copyright (C) 2016 Howard Hughes Medical Institute
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# command: _ssu-mask --pf 0.95 --pt 0.95 --afa --dna ssu_aln
# date:    Tue Sep 24 09:09:33 2019
#
# Masking alignments based on posterior probabilities...
#
#                                                     mask    
#                                                 ------------
# file name                  in/out  type  #cols  incl.  excl.
# -------------------------  ------  ----  -----  -----  -----
  ssu_aln.bacteria.stk        input   aln   1582      -      -
  ssu_aln.bacteria.mask      output  mask   1582    247   1335
  ssu_aln.bacteria.mask.pdf  output   pdf   1582    247   1335
  ssu_aln.bacteria.mask.afa  output   aln    247      -      -
#
# All attempts to draw structure diagrams of masks were successful.
#
# List of exec

In [13]:
#!cd $workDir; ~/seq_tools/seq_tools fasta_unwrap ssu_aln/ssu_aln.bacteria.mask.afa  | head

In [35]:
# !cd $workDir; cat sso_aln/sso_aln.bacteria.mask.afa ssu_aln/ssu_aln.bacteria.mask.afa > aln_for_tree_wSulfo.fasta
# !cd $workDir; cp ssu_aln/ssu_aln.bacteria.mask.afa  aln_for_tree.fasta
!cd $workDir; cp ssu_aln/ssu_aln.bacteria.mask.afa  aln_for_tree.fasta

In [36]:
!cd $workDir; head aln_for_tree.fasta

>ASV1
TACGAAGGGGGCTAGCGTTGTTCGGATTTACTGGGCGTAAAGCGCACGTAGGCGGACTAT
TAAGTCAGGGGTGAAATCCCGGGGCTCAACCCCGGAACTTTGATACTGGTAGTCTTGAGT
TCGAGAGAGGTGAGTGGAATTCCGAGTGTAGAGGTGAAATTCGTAGATATTCGGAGGAAC
ACCAGTGGCGAAGGCGGCTCACTGGCTCGACTGACGCTGAGGTGCGAAAGCGTGGGGAGC
AAACAGG
>ASV2
-ACATAGGTGGCAAACATTATCCGGAATTATTGGGCGTAAAGGGTGCGTAGGCGGCATGA
TAAGTTGCTGGTGGGAAATCAAGGCTCAACCTTGTGGTAGCAATACTGTCAAGCTAGAGG
GCAGAAGAGGTTAACGGAACTCTATGTGGAGCGGTAAAATGTGTAGATATATAGAAGAAC


## Inferring and rooting the tree

In [37]:
aln = LoadSeqs(os.path.join(workDir, 'aln_for_tree.fasta'), moltype=DNA)
t_unroot = build_tree_from_alignment(aln, moltype=DNA)

In [24]:
#t_rooted = t_unroot.rootedWithTip('X90478')

In [38]:
t_unroot.writeToFile(os.path.join(workDir, 'Master_unrooted.tree'))

# Tree with sulfo

In [27]:
aln = LoadSeqs(os.path.join(workDir, 'aln_for_tree_wSulfo.fasta'), moltype=DNA)
t_unroot = build_tree_from_alignment(aln, moltype=DNA)

In [28]:
t_rooted = t_unroot.rootedWithTip('X90478')

In [29]:
t_rooted.writeToFile(os.path.join(workDir, 'Master_wSulfo.tree'))